# 스크래핑한 데이터를 인공지능이 학습하도록 전처리합니다.

설치시 필요한 것
- konlpy : tokenize (품사태깅) 목적 -> 설치가 조금 어려워요...
- gensim : embedding (워드 임베딩) 목적


1. 품사 태깅
2. 워드 임베딩


In [1]:
from konlpy.tag import Twitter
import gensim
import tensorflow as tf

import numpy as np
import pandas as pd

D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as fp:
        data = [line.split('\t') for line in fp.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data

In [3]:
train_data = read_data("../data/ratings_train.txt")
test_data = read_data("../data/ratings_test.txt")

In [4]:
train_data[:5]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1']]

In [5]:
test_data[:5]

[['6270596', '굳 ㅋ', '1'],
 ['9274899', 'GDNTOPCLASSINTHECLUB', '0'],
 ['8544678', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '0'],
 ['6825595', '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '0'],
 ['6723715', '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??', '0']]

# 1. 품사 태깅

In [6]:
pos_tagger = Twitter()

def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
tokens = [tokenize(row[1]) for row in train_data]

In [8]:
train_data[:2]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']]

In [9]:
tokens[:2]

[['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 ['흠/Noun',
  '.../Punctuation',
  '포스터/Noun',
  '보고/Noun',
  '초딩/Noun',
  '영화/Noun',
  '줄/Noun',
  '..../Punctuation',
  '오버/Noun',
  '연기/Noun',
  '조차/Josa',
  '가볍다/Adjective',
  '않다/Verb']]

# 2. 워드 임베딩
 - min_alpha : training 됨에 따라 줄어든 learning_rate
 - alpha : initial learning rate
 - learning_rate decay 해줌
 

In [10]:
model = gensim.models.Word2Vec(size=300, sg = 1, alpha=0.025, min_alpha=0.025, seed=1234)
model.build_vocab(tokens)

for epoch in range(30):           
    model.train(tokens,model.corpus_count,epochs = model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


In [11]:
model.save("../data/word2vec.model")
model.most_similar('영화/Noun',topn = 20)  ## topn = len(model.wv.vocab)

D:\Anaconda3\envs\dev-tensorflow\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('../Punctuation', 0.9119613170623779),
 ('이/Josa', 0.8820630311965942),
 ('의/Josa', 0.8638195991516113),
 ('도/Josa', 0.863292932510376),
 ('는/Josa', 0.8534328937530518),
 ('것/Noun', 0.8118099570274353),
 ('만/Josa', 0.7741988301277161),
 ('내용/Noun', 0.7594603300094604),
 ('이다/Josa', 0.7448990345001221),
 ('평점/Noun', 0.7405540347099304),
 ('판타지영화/Noun', -0.32529205083847046),
 ('폰부스/Noun', -0.32803165912628174),
 ('만인/Noun', -0.33895355463027954),
 ('남아돌다/Verb', -0.33979544043540955),
 ('저예산영화/Noun', -0.3481564223766327),
 ('찍기/Noun', -0.35146015882492065),
 ('식스센스/Noun', -0.35187458992004395),
 ('대요/Noun', -0.3543408215045929),
 ('평정/Noun', -0.3548148274421692),
 ('과제/Noun', -0.35503286123275757)]